## Inference Operator PySDK E2E Expereience (JumpStart model)

<b>Prerequisite:</b> Data scientists should list clusters and set cluster context

In [ ]:
from sagemaker.hyperpod.hyperpod_manager import HyperPodManager

In [ ]:
HyperPodManager.list_clusters(region='us-east-2')

In [ ]:
# choose the HP cluster user works on
HyperPodManager.set_context('hp-cluster-for-inf-Beta2try1', region='us-east-2')

In [ ]:
# verify current kube context
HyperPodManager.get_context()

### Create JumpStart model endpoint

#### Create from spec object (for experienced users)

In [ ]:
from sagemaker.hyperpod.inference.config.hp_jumpstart_endpoint_config import Model, Server,SageMakerEndpoint, TlsConfig, EnvironmentVariables
from sagemaker.hyperpod.inference.hp_jumpstart_endpoint import HPJumpStartEndpoint
import yaml
import time

In [ ]:
# create configs
model=Model(
    model_id='deepseek-llm-r1-distill-qwen-1-5b',
    model_version='2.0.4',
)
server=Server(
    instance_type='ml.g5.8xlarge',
)
endpoint_name=SageMakerEndpoint(name='deepsek7bsme-testing-zhaoqi-0630-jumpstart')
tls_config=TlsConfig(tls_certificate_output_s3_uri='s3://tls-bucket-inf1-beta2')

# create spec
js_endpoint=HPJumpStartEndpoint(
    model=model,
    server=server,
    sage_maker_endpoint=endpoint_name,
    tls_config=tls_config,
)

In [ ]:
# use spec to deploy
js_endpoint.create()

In [ ]:
# poll status
t = 0
timeout = 600  # 600 seconds timeout  
interval = 15  # poll every 15 seconds

while t < timeout:
    # use refresh to fetch latest status
    js_endpoint.refresh()

    try:
        # deployment status will be available immediately
        deployment_status = js_endpoint.status.deploymentStatus.deploymentObjectOverallState
        if deployment_status== 'DeploymentFailed':
            print('Deployment failed!')
            break

        # endpoint status will appear be available from refresh() at some point
        endpoint_status = js_endpoint.status.endpoints.sagemaker.state
        if endpoint_status == 'CreationCompleted':
            print('Endpoint is available!')
            break
    except:
        pass

    time.sleep(interval)
    t += interval

if t >= timeout:
    print('Endpoint creation timed out!')

In [ ]:
# print endpoint in yaml
def print_yaml(endpoint):
    print(yaml.dump(endpoint.model_dump(exclude_none=True)))

<b>Note:</b> We auto-generate config class definitions above using script, such as `Model`, `Server`, `SageMakerEndpoint`. This is based on [Inference CRD file](https://code.amazon.com/packages/AWSCrescendoInferenceOperator/blobs/mainline/--/dist/config/crd/inference.sagemaker.aws.amazon.com_jumpstartmodels.yaml).

In [ ]:
# output is similar to kubectl get jumpstartmodels
endpoint_list = HPJumpStartEndpoint.list()
print_yaml(endpoint_list[0])

In [ ]:
# output is similar to kubectl describe jumpstartmodel huggingface-eqa-bert-base-cased
endpoint = HPJumpStartEndpoint.get(name='deepseek-llm-r1-distill-qwen-1-5b')
print_yaml(endpoint)

### Invoke endpoint

In [ ]:
# invoke
data='{"inputs":"What is the capital of USA?"}'

endpoint.invoke(body=data).body.read()

In [ ]:
# delete endpoint
endpoint.delete()